In [ ]:
from policy.dataset.ms2CONet import ManiSkillDataset, ManiSkillConvONetDataset_V2
from policy import config
import numpy as np
import trimesh
import torch

cfg = config.load_config("assets/perception/conv_onet_test.yaml")

dataset = ManiSkillDataset(cfg["data"]["dataset"])
cam2world = dataset[0]["obs"]["camera_param"]["base_camera"]["cam2world_gl"][0]
groundtruth = ManiSkillDataset(cfg["data"]["ground_truth"])
inputs = ManiSkillConvONetDataset_V2(cfg["data"]["dataset"], cfg["data"]["ground_truth"], [0])

In [ ]:
def sapien_pose_to_opencv_extrinsic(sapien_pose_matrix: np.ndarray) -> np.ndarray:
    sapien2opencv = np.array(
        [
            [0.0, -1.0, 0.0, 0.0],
            [0.0, 0.0, -1.0, 0.0],
            [1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1.0],
        ],
        dtype=np.float32,
    )
    ex = sapien2opencv @ np.linalg.inv(sapien_pose_matrix) # world -> camera

    return ex

cam_ex = dataset[0]["obs"]["camera_param"]["base_camera"]["extrinsic_cv"][0]

def invert_transform(H: np.ndarray):
    assert H.shape[-2:] == (4, 4), H.shape
    H_inv = H.copy()
    R_T = np.swapaxes(H[..., :3, :3], -1, -2)
    H_inv[..., :3, :3] = R_T
    H_inv[..., :3, 3:] = -R_T @ H[..., :3, 3:]
    return H_inv


In [ ]:
cam0 = 16384
cam1 = 16384*2
cam2 = 16384*3
cam3 = 16384*4
 
# print(cam2world)
points = torch.asarray(groundtruth[0]["obs"]["pointcloud"]["xyzw"][0, 0:cam0, :])
# cam_ex = sapien_pose_to_opencv_extrinsic(cam_ex)
inv_cam_ex = invert_transform((np.asarray(cam_ex)))

points = points @ torch.tensor(inv_cam_ex)
colors = groundtruth[0]["obs"]["pointcloud"]["rgb"][0, 0:cam0, :]
trans_pcd = trimesh.points.PointCloud(points[..., 0:3], colors=colors)

data = inputs.datamap[(0, 0, 0)]["input_data"]
input_points = data[:, 0:3]
input_colors = data[:, 3:] * 255.0
ones = torch.ones((16384, 1))

input_points = torch.cat((input_points, ones), dim=1).float()
input_colors = torch.cat((input_colors, ones), dim=1).int()
# input_points = input_points @ torch.tensor(inv_c2w)

input_pcd = trimesh.points.PointCloud(input_points[..., 0:3], input_colors)

scene = trimesh.Scene([input_pcd, trans_pcd])
scene.show()

In [ ]:
from policy.dataset.ms2CONet import get_MS_loaders
from policy import config

cfg = config.load_config("assets/perception/conv_onet_test.yaml")
# dataset = ManiSkillConvONetDataset_V2(cfg["data"]["dataset"], indices=[0])
model = config.get_model(cfg)
# print(dataset)
train_loader, val_loader = get_MS_loaders(cfg)

In [ ]:
import trimesh
import torch
import numpy as np

data = train_loader.dataset[-10]
cam2world = dataset[0]["obs"]["camera_param"]["base_camera"]["cam2world_gl"][0]

# input visualization 
input_points = data["input_data"][:, 0:3]
print(input_points.shape)
input_colors = data["input_data"][:, 3:] * 255.0
ones = torch.ones((16384, 1))
input_colors = torch.cat((input_colors, ones), dim=1).int()

xyzw = torch.cat([input_points, ones != 0], dim=-1).reshape(
                input_points.shape[0], -1, 4) @ cam2world.transpose(1, 2)


# rot_mat1 = torch.tensor([[np.cos(np.pi/2), -np.sin(np.pi/2), 0],
#                         [np.sin(np.pi/2), np.cos(np.pi/2), 0],
#                         [0, 0, 1]]).float()

# rot_mat2 = torch.tensor([
#                         [np.cos(np.pi),0, -np.sin(np.pi)],
#                         [0, 1, 0],
#                         [np.sin(np.pi), 0, np.cos(np.pi)]
#                         ]).float()

# refl_mat = torch.tensor([[-1, 0, 0],
#                          [0, 1, 0],
#                          [0, 0, -1]]).float()

# input_points = torch.mm(input_points, refl_mat)
# input_points = torch.mm(input_points, rot_mat1)
# input_points = torch.mm(input_points, rot_mat2)


input_pcd = trimesh.points.PointCloud(input_points, input_colors)
input_scene = input_pcd.scene()

# gt visualization
gt_points = data["gt_points"]
print(gt_points.shape)
gt_colors = data["gt_colors"]
gt_colors = torch.cat((gt_colors, ones), dim=1).int()

gt_pcd = trimesh.points.PointCloud(gt_points, gt_colors)
gt_scene = gt_pcd.scene()

scene = trimesh.Scene([input_pcd])
scene.show()



In [1]:
from policy.dataset.ms2CONet import ManiSkillDataset, ManiSkillConvONetDataset_V3, get_MS_loaders_V3
from policy import config
from policy.perception.conv_onet.training import Trainer
import numpy as np
import trimesh
import torch

cfg = config.load_config("assets/perception/conv_onet_test.yaml")
# data = ManiSkillConvONetDataset_V3(cfg["data"]["dataset"], [0])
train_loader, val_loader = get_MS_loaders_V3(cfg)

Training Indices: [1, 0]
Validation Indices: [2]


152it [00:11, 13.58it/s]
137it [00:10, 13.48it/s]
172it [00:12, 13.76it/s]

Shuffling: True


In [2]:
# inputs = data[0]["inputs"].unsqueeze(0)
# points = data[0]["points"].unsqueeze(0)
# print(points.shape)
# print(inputs.shape)

model = config.get_model(cfg)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
trainer = Trainer(model, optimizer, None)

In [3]:
# next(iter(train_loader))
#  trainer.compute_loss(data[0])
trainer.train_step(next(iter(train_loader)))

103679.265625

In [7]:
val_test = next(iter(val_loader))

In [8]:
trainer.eval_step(val_test)

AttributeError: 'NoneType' object has no attribute 'to'